In [1]:
# Some imports
import sys
import os, glob
from mfx.processlocalizations import ProcessLocalizations
import re
from matplotlib import pyplot as plt
import math
import numpy as np
from mfx import mfxcolnames as col

In [2]:
# Get file names

OUTDIR_LOC = 'C:/Users/apoliti/Desktop/mflux_zarr_tmp_storage/analysis/' # Main directory to store zarr files
OUTDIR_REM =  'Z:/siva_minflux/analysis/'  # Main directory to store results 
INDIR  = 'Z:/siva_minflux/data/'       # main directory of msr file

# Multiple washes with different imager strand
indir_mwash = {'Syp_ATG9': INDIR + '/Multiwash/Syp_ATG9/', 
             'ZnT3_Syp': INDIR + '/Multiwash/ZnT3_Syp/'}
outdir_mwash = {'Syp_ATG9': OUTDIR_REM +  '/Multiwash/Syp_ATG9/', 
                'ZnT3_Syp': OUTDIR_REM +  '/Multiwash/ZnT3_Syp/'}
zarrdir_mwash = {'Syp_ATG9': OUTDIR_LOC + '/Multiwash/Syp_ATG9', 
                'ZnT3_Syp': OUTDIR_LOC + '/Multiwash/ZnT3_Syp/'}

# Wash with a single imager strand
indir_swash = {'Syp': INDIR + '/Single wash/Syp/', 
             'ATG9': INDIR + '/Single wash/ATG9/'}
outdir_swash = {'Syp': OUTDIR_REM +  '/Single wash/Syp/', 
                'ATG9': OUTDIR_REM +  '/Single wash/ATG9/'}
zarrdir_swash = {'Syp': OUTDIR_LOC +  '/Single wash/Syp/', 
                'ATG9': OUTDIR_LOC +  '/Single wash/ATG9/'}


# Consitency controls. Wash with a single imager strand but multiple times. 
indir_cwash = {'VGLUT1_VGLUT1': INDIR + '/Multiwash/VGLUT1_VGLUT1/'}
outdir_cwash = {'VGLUT1_VGLUT1': OUTDIR_REM +  '/Multiwash/VGLUT1_VGLUT1/'}
zarrdir_cwash = {'VGLUT1_VGLUT1': OUTDIR_LOC +  '/Multiwash/VGLUT1_VGLUT1/'}



npy_mwash = {'Syp_ATG9': [], 'ZnT3_Syp': []}

for key in outdir_mwash:
    for (root, dirs, files) in os.walk(outdir_mwash[key]):
        if len(dirs) > 0:
            for adir in dirs:
                npyfile = os.path.join(root, adir, adir +'.npy')
                if os.path.exists(npyfile):
                    npy_mwash[key].append(npyfile)
                    

npy_swash = {'Syp': [], 'ATG9': []}

for key in outdir_swash:
    for (root, dirs, files) in os.walk(outdir_swash[key]):
        if len(dirs) > 0:
            for adir in dirs:
                npyfile = os.path.join(root, adir, adir +'.npy')
                if os.path.exists(npyfile):
                    npy_swash[key].append(npyfile)

npy_cwash = {'VGLUT1_VGLUT1': []}

for key in outdir_cwash:
    for (root, dirs, files) in os.walk(outdir_cwash[key]):
        if len(dirs) > 0:
            for adir in dirs:
                npyfile = os.path.join(root, adir, adir +'.npy')
                if os.path.exists(npyfile):
                    npy_cwash[key].append(npyfile)
                    

In [ ]:
pl = ProcessLocalizations(npy_cwash['VGLUT1_VGLUT1'][1])
pl.log_parameters()
pl.MIN_LOCALIZATIONS = 3
pl.trim_min_localizations()
# Split tracks if needed, remove outliers in each track
pl.cluster_tid(method=pl.CLS_METHOD_BAYES_GMM)

# find objects that belong to the same cluster according to distance 30 nm
pl.cluster_meas(method=pl.CLS_METHOD_DBSCAN)
#pl.DBCLUSTER_EPS_ALL = 6e-8
pl.cluster_all(method=pl.CLS_METHOD_DBSCAN)
pl.cluster_all_intersect(col.CLS_ALL)

2022-09-21 18:29:26,839 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/VGLUT1_VGLUT1/220811_VGLUT1_ROI01_First\220811_VGLUT1_ROI01_First.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 2e-08
DBCLUSTER_EPS_ALL 2e-08
DBCLUSTER_EPS_MERGED_ALL 2e-08
DBCLUSTER_EPS_MERGED_MEAS 2e-08

2022-09-21 18:29:26,839 [INFO] **** log_parameters ****
file_path Z:/siva_minflux/analysis//Multiwash/VGLUT1_VGLUT1/220811_VGLUT1_ROI01_First\220811_VGLUT1_ROI01_First.npy
MIN_LOCALIZATIONS 3
FACT_SPLIT_LOCALIZATIONS 2
DBCLUSTER_SIZE 3
DBCLUSTER_EPS_TRACK 2e-08
DBCLUSTER_EPS_MEAS 2e-08
DBCLUSTER_EPS_ALL 2e-08
DBCLUSTER_EPS_MERGED_ALL 2e-08
DBCLUSTER_EPS_MERGED_MEAS 2e-08

2022-09-21 18:29:26,858 [INFO] **** trim_min_localizations ****
220811_VGLUT1_P1 Removed 927/2692 tracks with less than 3 localizations
220811_VGLUT1_P2 Removed 738/1813 tracks with less than 3 localizations

2022-09-21 18:29:26,858 [INFO] **** trim_

In [10]:
pl.DBCLUSTER_EPS_ALL = 3e-8
pl.cluster_all(method=pl.CLS_METHOD_DBSCAN)
pl.cluster_all_intersect(col.CLS_ALL)

*******cluster_all*******
220811_VGLUT1_P1, Method: dbscan, eps: 30.00 nm
Total tracks TID2: 1805, total cluster all: 457
*******cluster_all*******
220811_VGLUT1_P2, Method: dbscan, eps: 30.00 nm
Total tracks TID2: 1098, total cluster all: 376
*******cluster_all_intersect*******
220811_VGLUT1_P1 cluster with 220811_VGLUT1_P2 cls 74/456 = 0.16; TID2 1212/1805 = 0.67
220811_VGLUT1_P2 cluster with 220811_VGLUT1_P1 cls 74/375 = 0.20; TID2 673/1098 = 0.61


In [ ]:
sum(pl.loc['220825_Syp_P1'][col.TID2] == -1)

In [ ]:
npy_cwash['VGLUT1_VGLUT1'][1]